In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import random
import crossval #this is the file Eli wrote
import imblearn



from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier, BaggingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score, plot_precision_recall_curve, precision_recall_curve, precision_score, recall_score

from sklearn.model_selection import RandomizedSearchCV, train_test_split
import shap
import lightgbm as lgb 

pd.set_option('max_colwidth', 80)

In [2]:
columns = ['TARGET_FID', 'Length', 'Install_year', 'Slope', 'Width', 'Break_Yr',
       'Target', 'AGE',
       'Process_year']

In [3]:
df = pd.read_csv('/Users/nelsschimek/Documents/IIDC/Water Pipe Project/Water-Pipe-Project/Water-Pipe-Project/data/final_data_5yr.csv') 
df.head()

,TARGET_FID,Length,Install_year,Slope,Width,Break_Yr,Target,AGE,Nearby_breaks_1yr,Nearby_breaks_2yr,Nearby_breaks_3yr,Nearby_breaks_4yr,Nearby_breaks_5yr,Process_year
0,1,314.79,1972,0.0,8.0,NaN,0,42,0,0,0,0,0,2014
1,2,363.39,1972,4.0,8.0,NaN,0,42,0,0,0,0,0,2014
2,3,323.51,1972,4.0,8.0,NaN,0,42,0,0,0,0,0,2014
3,4,329.13,1928,6.0,12.0,NaN,0,86,0,0,0,0,0,2014
4,5,273.64,1928,4.0,18.0,NaN,0,86,0,0,0,0,0,2014


In [4]:
param_list = {
      'n_estimators': [50],
      'max_depth': [30]
    } 
param = {}
for key in param_list.keys():
    val = random.choice(param_list[key])
    param[key] = val

In [5]:
# Used to build a list containing dictionaries w every compination of params
# from param list, start with a list containing one dict (src), 
# then call for each param (cross, keyword)
def cross_params(src, cross, keyword):
    combos = []
    for d in src:
        for val in cross:
            cpy = d.copy()
            cpy[keyword] = val
            combos.append(cpy)
    return combos

# Uses cross_params to get list of all parameter combinations 
def get_param_combos(param_list):
    param_combos = [{}]
    for kw in param_list:
        param_combos = cross_params(param_combos, param_list[kw], kw)
    return param_combos

In [6]:
def rand_param_search(data, param_list, classifier_fn, n_val=2, threshold=0.1, print_results=True):
    """

    """
    best_score = -1.0 
    best_param = {}
    param_combos = get_param_combos(param_list)
    for param in param_combos:
        print("using the following parameters:")
        print(param)
        avgs = crossval.crossval_by_year(data, classifier_fn, param, n_val, threshold, print_results, under_sample=True)
        print(avgs[0])
        print(avgs[1])
        print(avgs[2])
        print()
        if avgs[0] > best_score:
            best_param = param
            best_score = avgs[0]
    
    
    return best_param, best_score

In [7]:
best_score, best_param = rand_param_search(df[columns], param_list, RandomForestClassifier)
print(best_param)
print(best_score)

using the following parameters:
{'n_estimators': 50, 'max_depth': 30}
Cross-validation iteration 1 of 7 results:
Balanced accuracy = 0.5044
Recall            = 1.0000
Precision         = 0.0043

Training set balanced accuracy = 1.0
Train Recall            = 1.0000
Train Precision         = 1.0000

Cross-validation iteration 2 of 7 results:
Balanced accuracy = 0.5023
Recall            = 1.0000
Precision         = 0.0047

Training set balanced accuracy = 1.0
Train Recall            = 1.0000
Train Precision         = 1.0000



KeyboardInterrupt: 

In [ ]:
print(best_param)
print(best_score)

0.7422383115676469
{'n_estimators': 50, 'max_depth': 30}
